## Import Classes and Files

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd
#need these libraries

In [2]:
hits16 = pd.read_csv('2016 Hitting.csv')
pitch16 = pd.read_csv('2016 Pitching.csv')

hits16.index = hits16.iloc[:,0]
pitch16.index = pitch16.iloc[:,0]

In [3]:
hits17 = pd.read_csv('2017 Hitting.csv')
pitch17 = pd.read_csv('2017 Pitching.csv')

hits17.index = hits17.iloc[:,0]
pitch17.index = pitch17.iloc[:,0]

In [4]:
hits18 = pd.read_csv('2018 Hitting.csv')
pitch18 = pd.read_csv('2018 Pitching.csv')

hits18.index = hits18.iloc[:,0]
pitch18.index = pitch18.iloc[:,0]

In [5]:
hits19 = pd.read_csv('2019 Hitting.csv')
pitch19 = pd.read_csv('2019 Pitching.csv')

hits19.index = hits19.iloc[:,0]
pitch19.index = pitch19.iloc[:,0]

In [6]:
hits21 = pd.read_csv('2021 Hitting.csv')
pitch21 = pd.read_csv('2021 Pitching.csv')

hits21.index = hits21.iloc[:,0]
pitch21.index = pitch21.iloc[:,0]

#past data

In [7]:
hits22 = pd.read_csv('2022 Hitting.csv')
pitch22 = pd.read_csv('2022 Pitching.csv')

hits22.index = hits22.iloc[:,0]
pitch22.index = pitch22.iloc[:,0]


In [8]:
lineups22 = pd.read_csv('2022 Lineups.csv')
lineups22.index = lineups22.iloc[:,0]

#default lineups for each team from the year 2022, will need daily lineups edited in the future

In [12]:
#start22 = pd.read_csv('2022 Starter Projections.csv')
#relief22 = pd.read_csv('2022 Reliever Projections.csv')
#hit22 = pd.read_csv('2022 Hitting Projections.csv')


#start22.index = start22.iloc[:,0]
#relief22.index = relief22.iloc[:,0]
#hit22.index = hit22.iloc[:,0]

#2022 player projections

In [9]:
games21 = pd.read_table('GL2021.txt',delimiter=',',usecols=[0,3,6,9,10,102,104],header=None)
games21.columns= ['Date','Visiting Team', 'Home Team', "Visitor Score", 'Home Score', 'Visiting Pitcher', 'Home Pitcher']

games19 = pd.read_table('GL2019.txt',delimiter=',',usecols=[0,3,6,9,10,102,104],header=None)
games19.columns= ['Date','Visiting Team', 'Home Team', "Visitor Score", 'Home Score', 'Visiting Pitcher', 'Home Pitcher']

games18 = pd.read_table('GL2018.txt',delimiter=',',usecols=[0,3,6,9,10,102,104],header=None)
games18.columns= ['Date','Visiting Team', 'Home Team', "Visitor Score", 'Home Score', 'Visiting Pitcher', 'Home Pitcher']

games17 = pd.read_table('GL2017.txt',delimiter=',',usecols=[0,3,6,9,10,102,104],header=None)
games17.columns= ['Date','Visiting Team', 'Home Team', "Visitor Score", 'Home Score', 'Visiting Pitcher', 'Home Pitcher']

games16 = pd.read_table('GL2016.txt',delimiter=',',usecols=[0,3,6,9,10,102,104],header=None)
games16.columns= ['Date','Visiting Team', 'Home Team', "Visitor Score", 'Home Score', 'Visiting Pitcher', 'Home Pitcher']

In [10]:
hits18['OBPxS'].loc['Mike Trout']

#It works! but the below values are average OBPxS for every player's projection and includes a lot of players who likely won't play

#2016 OBPxS average: 0.108266
#2017 OBPxS average:0.110482
#2018 OBPxS average:0.112843
#2019 OBPxS average:0.108721
#2021 OBPxS average:0.111083
#2022 OBPxS average:0.108459

0.24839

## The Calculations

In [11]:

#ORIGINAL VERSION, FOR REFERENCE
def game_calc(home_team,away_team,homeP,awayP,pitching=pitch16,hitting=hits16,xRuns=8.6): 
    #starter references starting pitcher stats for a given year
    if (pitching['Tm'].loc[homeP] != home_team) & (pitching['Tm'].loc[homeP] != '- - -'): 
        #print("Home Starter doesn't play for that team")
        #using a mismatched starter/team will probably break everything, so let's not do that
        return [0,0,.501,.499]
    elif (pitching['Tm'].loc[awayP] != away_team) & (pitching['Tm'].loc[awayP] != '- - -'):
        #print("Away Starter doesn't play for that team")
        return [0,0,.501,.499]
    else:
        away_xRA = pitching['Combo'].loc[awayP]*(2/3)   +  (4.53*(1/3))*relief['League Percentile'].loc[away_team] #starter + relief xRA
        home_xRA = pitching['Combo'].loc[homeP]*(2/3)   +  (4.53*(1/3))*relief['League Percentile'].loc[home_team]
        home_xRS = hitting['League Percentile'].loc[home_team] * (away_xRA/4.53) * 4.53
        away_xRS = hitting['League Percentile'].loc[away_team] * (home_xRA/4.53) * 4.53
        
        if xRuns == 8.6: #I don't really want to use the default value of 2*average runs scored/game but it's there if needed
                xRuns = home_xRS+away_xRS
        
        exponent = 1.5*math.log(xRuns,10) + .45
        
        home_win_pct = (home_xRS**exponent)/((away_xRS**exponent) + (home_xRS**exponent)) + .03 #3% home field adjustment
        away_win_pct = (away_xRS**exponent)/((home_xRS**exponent) + (away_xRS**exponent)) - .03
        
        
    
        return [home_xRS, away_xRS, home_win_pct, away_win_pct]
    

In [20]:
#this function will eventually return the full game preedicted results

def lineup_game_calc(home_team,away_team,homeP,awayP,home_lineup=lineups22.loc['ATL'][1:10], away_lineup = lineups22.loc['ARI'][1:10], pitching=pitch22,hitting=hits22,xRuns=8.6):
    if (pitching['Tm'].loc[homeP] != home_team) & (pitching['Tm'].loc[homeP] != '- - -'): 
        #print("Home Starter doesn't play for that team")
        #using a mismatched starter/team will probably break everything, so let's not do that
        return [0,0]
    elif (pitching['Tm'].loc[awayP] != away_team) & (pitching['Tm'].loc[awayP] != '- - -'):
        #print("Away Starter doesn't play for that team")
        return [0,0]
    else:
        
        #2022 average O rating = 1.30927
        
        home_o_rating = lineup_strength(home_team,lineups22,hitting)
        away_o_rating = lineup_strength(away_team,lineups22,hitting)
        print(home_o_rating,away_o_rating)
        #offensive rating is a calculated stat based on every players OBPxS and how many plate appearances they'll see
        #need to calculate relief pitching somehow
        
        away_xRA = pitching['ERA'].loc[awayP]*(pitching['IP']/pitching['GS'])   +  (4.53*(1/3))*relief['League Percentile'].loc[away_team]
        #original- away_xRA = starter['Combo'].loc[awayP]*(2/3)   +  (4.53*(1/3))*relief['League Percentile'].loc[away_team]
    
    
        return [home_o_rating,away_o_rating]

In [21]:
lineup_game_calc('NYM','LAD','Max Scherzer', 'Clayton Kershaw',lineups22.loc['NYM'][1:10],lineups22.loc['LAD'][1:10],pitch22,hits22)

[1.3247623981199999, 1.5643957150899999]

In [38]:
Atlanta = lineups22.loc['ATL'][1:10]
for i in Atlanta:
    print(i,hits22['OBPxS'].loc[i])
#testing and visualizing lineups

Eddie Rosario 0.1457
Ozzie Albies 0.16104
Matt Olson 0.20925
Marcell Ozuna 0.154512
Austin Riley 0.166815
Adam Duvall 0.141899
Dansby Swanson 0.138346
Alex Dickerson 0.139797
Travis d'Arnaud 0.124949


In [15]:
i=0
test_team_o_rating = 0
#team.size should always be 9
while i < Atlanta.size:
    #modifier for different lineup position
    mod_list=[1.102,1.076,1.0496,1.026,1,0.9787,0.9504,00.92199,0.8936]
    
    test_team_o_rating+=(hits22['OBPxS'].loc[Atlanta[i]]*mod_list[i])
        
    i+=1
test_team_o_rating

1.38972000413

In [16]:
# this function returns a team's offensive rating

def lineup_strength(team, season_lineup = lineups22, season_stats = hits22):
    #team is in 3 letter 'OAK' format
    mod_list=[1.102,1.076,1.0496,1.026,1,0.9787,0.9504,00.92199,0.8936]
    k=0
    #k is just the iterant variable
    
    team_o_rating=0
    #team_o_rating, when completed is the team's full lineup strength
    
    this_lineup=season_lineup.loc[team][1:10]
    #specifying which team's lineup
    
    while k <9:
        team_o_rating+=(season_stats['OBPxS'].loc[this_lineup[k]]*mod_list[k])
        k+=1
        #iterate through lineup and add the batting order adjusted player impacts
    return team_o_rating
        


In [50]:
def calc_league_offense(season_lineup = lineups22, season_hitting = hits22):
    #using this to find our league-wide offensive benchmark
    league_rating = 0
    
    for t in lineups['Team']:
        rating = lineup_strength(t,season_lineup ,season_hitting)
        
        league_rating += rating
        #adding each team's offensive power to the league as a whole
    league_rating = league_rating/30
    #should give league average OBPxS?
    return league_rating
#league wide offense is calculated, giving us a baseline.

In [49]:
calc_league_offense()

1.309273464069333

In [37]:
for t in lineups['Team']:
    print(t)

ARI
ATL
BAL
BOS
CHC
CHW
CIN
CLE
COL
DET
HOU
KCR
LAA
LAD
MIA
MIL
MIN
NYM
NYY
OAK
PHI
PIT
SDP
SEA
SFG
STL
TBR
TEX
TOR
WAS


In [ ]:
 def new_calc(home_team, away_team, homeP, awayP, home_lineup = lineups.loc[home_team][1:10], away_lineup = lineups.loc[away_team][1:10])
    
    away_innings =start22['InningsPG'].loc[awayP]
    away_xRA = start22['Combo'].loc[awayP] * (away_innings / 9) 
    
    